# Phase 2

### Imports

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

### Reading Data

In [6]:
spark = SparkSession.builder.getOrCreate()

hotel_booking = spark.read.csv(
    "../phase1_output_datasets/hotel_booking_cleaned.csv",
    header=True,
    inferSchema=True,
)
customer_reservations = spark.read.csv(
    "../phase1_output_datasets/customer_reservations.csv",
    header=True,
    inferSchema=True,
)
all_bookings = spark.read.csv(
    "../phase1_output_datasets/all_bookings.csv",
    header=True,
    inferSchema=True,
)

In [12]:
cols_to_check = [c for c in all_bookings.columns if c not in ("email", "booking_id")]

all_bookings = all_bookings.na.drop(subset=cols_to_check)

## 2.1 Spark Analysis

### Cancellation rates
Calculate cancellation rates for each month

In [ ]:
cancellation_rates = (all_bookings
    .groupBy("arrival_year", "arrival_month")
    .agg(F.avg("booking_status").alias("cancellation_rate"))
    .orderBy("arrival_year", "arrival_month"))

cancellation_rates.show()

+------------+-------------+-------------------+
|arrival_year|arrival_month|  cancellation_rate|
+------------+-------------+-------------------+
|        2015|          7.0| 0.4554670528602462|
|        2015|          8.0|0.41217436161980914|
|        2015|          9.0| 0.4087332808811959|
|        2015|         10.0| 0.3488513925594633|
|        2015|         11.0|0.20948275862068966|
|        2015|         12.0| 0.3355172413793103|
|        2016|          1.0| 0.2514672686230248|
|        2016|          3.0| 0.3082706766917293|
|        2016|          4.0| 0.3810140636565507|
|        2016|          5.0| 0.3503476033662642|
|        2016|          6.0|0.39678030303030304|
|        2016|          7.0|0.32798769771528996|
|        2016|          8.0| 0.3609022556390977|
|        2016|          9.0| 0.3756269738064276|
|        2016|         10.0| 0.4067357512953368|
|        2016|         11.0| 0.3686823104693141|
|        2016|         12.0|0.36311389759665624|
|        2017|      

### Averages
Compute average price and average number of nights for each month

In [21]:
avg_month_price = (all_bookings
    .groupBy("arrival_year", "arrival_month")
    .agg(F.avg("avg_price_per_room").alias("avg_price_per_month"))
    .orderBy("arrival_year", "arrival_month"))

avg_month_price.show()

+------------+-------------+-------------------+
|arrival_year|arrival_month|avg_price_per_month|
+------------+-------------+-------------------+
|        2015|          7.0|   97.8634829833452|
|        2015|          8.0| 105.77794686613359|
|        2015|          9.0|  94.88720889063715|
|        2015|         10.0|  79.07416751372217|
|        2015|         11.0|  60.76284051724128|
|        2015|         12.0|  74.30403448275867|
|        2016|          1.0|  65.15584198645607|
|        2016|          3.0|  79.36027777777778|
|        2016|          4.0|  89.12324204293117|
|        2016|          5.0|   96.4782821075742|
|        2016|          6.0| 107.03919507575752|
|        2016|          7.0|  125.4895210896312|
|        2016|          8.0| 142.85367629600327|
|        2016|          9.0| 114.73706669143604|
|        2016|         10.0|   95.2871664507771|
|        2016|         11.0|  80.93624774368196|
|        2016|         12.0|  86.56064263322848|
|        2017|      

In [ ]:
avg_nights = (all_bookings
    .withColumn(
        "total_nights",
        F.col("stays_in_weekend_nights") + F.col("stays_in_week_nights")
    )
    .groupBy("arrival_year", "arrival_month")
    .agg(F.avg("total_nights").alias("avg_nights_per_month"))
    .orderBy("arrival_year", "arrival_month"))

avg_nights.show()

+------------+-------------+--------------------+
|arrival_year|arrival_month|avg_nights_per_month|
+------------+-------------+--------------------+
|        2015|          7.0|  3.8664011585807385|
|        2015|          8.0|  3.6793912819190093|
|        2015|          9.0|   3.466365066876475|
|        2015|         10.0|   3.062614352510673|
|        2015|         11.0|              3.2875|
|        2015|         12.0|                3.05|
|        2016|          1.0|  2.7598194130925506|
|        2016|          3.0|  3.2439431913116126|
|        2016|          4.0|  3.2041080680977054|
|        2016|          5.0|  3.2111233077204537|
|        2016|          6.0|   3.422727272727273|
|        2016|          7.0|   3.935413005272408|
|        2016|          8.0|  3.9066086268302334|
|        2016|          9.0|   3.457737321196359|
|        2016|         10.0|  3.1863665803108807|
|        2016|         11.0|   3.148691335740072|
|        2016|         12.0|   3.375130616509927|


### Monthly Bookings
Count monthly bookings by market segment. In categories, the term TA means Travel Agents and TO means Tour Operators

In [ ]:
# Online TA
# Offline TA/TO

bookings_by_market = (all_bookings
    .groupBy("market_segment_type")
    .agg(F.count("*").alias("num_bookings"))
    .orderBy(F.desc("num_bookings")))

bookings_by_market.show()

+-------------------+------------+
|market_segment_type|num_bookings|
+-------------------+------------+
|             Online|       55477|
|            Offline|       28177|
|             Groups|       13375|
|             Direct|        7335|
|          Corporate|        5212|
|      Complementary|         886|
|           Aviation|         251|
|          Undefined|           2|
+-------------------+------------+



### Seasonality
Identify the most popular month of the year for bookings based on revenue

In [30]:
avg_month_price.orderBy(F.desc("avg_price_per_month")).show()

+------------+-------------+-------------------+
|arrival_year|arrival_month|avg_price_per_month|
+------------+-------------+-------------------+
|        2016|          8.0| 142.85367629600327|
|        2016|          7.0|  125.4895210896312|
|        2018|          9.0| 122.90583051991898|
|        2018|          8.0| 121.21475884244347|
|        2018|          7.0| 115.44809151349233|
|        2016|          9.0| 114.73706669143604|
|        2018|          5.0| 113.81067744418755|
|        2018|          6.0| 111.96650015610335|
|        2018|         10.0|  111.4457226792006|
|        2016|          6.0| 107.03919507575752|
|        2015|          8.0| 105.77794686613359|
|        2017|          9.0| 103.33518496058223|
|        2018|          4.0| 101.99880116959012|
|        2015|          7.0|   97.8634829833452|
|        2018|         11.0|  96.53972138877005|
|        2016|          5.0|   96.4782821075742|
|        2016|         10.0|   95.2871664507771|
|        2018|      

In [31]:
avg_month_price.orderBy(F.desc("avg_price_per_month")).show(1)

+------------+-------------+-------------------+
|arrival_year|arrival_month|avg_price_per_month|
+------------+-------------+-------------------+
|        2016|          8.0| 142.85367629600327|
+------------+-------------+-------------------+
only showing top 1 row
